# REDIS SEARCH - GETTING STARTED

# 1 - Introduction

In [1]:
import redis
import json
from redis.commands.json.path import Path
import warnings
warnings.filterwarnings('ignore')

In [2]:
# CREATE CLIENT CONNECTION TO DATABASE, TEST IT.
client= redis.StrictRedis(
# REPLACE THE FIELDS BELOW WITH YOUR REDIS DATABASE INFO    
    host = 'redis-12553.c238.us-central1-2.gce.cloud.redislabs.com', 
    port = 12553,
    password='<YOUR DATABASE PASSWORD>', 

    decode_responses=True,
    socket_timeout=None)

print(client.set('Hello', 'Hello from Redis - redis-search'))   # insert key in Redis
print(client.get('Hello'))                                     # read from Redis

True
Hello from Redis - redis-search


In [ ]:
# FLUSH DATABASE (when necessary)
#client.execute_command('Flushdb')

# 1.1 - Basic JSON Operations - CRUD

In [3]:
# CREATE
result=client.execute_command('JSON.SET','name','$','"John"')

# READ
result = client.execute_command('JSON.GET', 'name')
print(result)

"John"


In [4]:
# UPDATE
client.execute_command('JSON.SET','name','$','"Lucy"')

result=client.execute_command('JSON.GET', 'name')
print(result)

"Lucy"


In [5]:
# DELETE
client.execute_command('JSON.DEL','name')

result = client.execute_command('JSON.GET', 'name')
print(result)

None


# 1.2 - Multiple Properties and Different Data Types

In [6]:
# SET MULTIPLE PROPERTIES

client.execute_command('JSON.SET','ex1:1', '$', '{"field1": "val1", "field2": "val2"}')
result = client.execute_command('JSON.GET','ex1:1')
print(result)

{"field1":"val1","field2":"val2"}


In [7]:
# SET DIFFERENT DATA TYPES

client.execute_command('JSON.SET', 'ex1:2','$','{"field1":"val1","field2":2,"field3":true, "field4": null,"field5":false}')
result = client.execute_command('JSON.GET','ex1:2')
print(result)

{"field1":"val1","field2":2,"field3":true,"field4":null,"field5":false}


# 1.3 - JSON Arrays and Objects

In [8]:
# SET AN ARRAY

client.execute_command('JSON.SET','ex1:3', '$', '{"arr1": ["val1", "val2", "val3"]}')
result = client.execute_command('JSON.GET', 'ex1:3')
print(result)

{"arr1":["val1","val2","val3"]}


In [9]:
# SET AN JSON OBJECT

client.execute_command('JSON.SET','ex1:4', '$', '{"obj1": {"str1": "val1", "num2": 2}}')
result = client.execute_command('JSON.GET', 'ex1:4')
print(result)

{"obj1":{"str1":"val1","num2":2}}


# 1.4 - JSON Object With a Mix of Property Data Types

In [10]:
# SET MULTIPLE FIELDS, RETRIEVE FULL OBJECT

client.execute_command('JSON.SET','ex1:5','$',
'{"str1":"val1","arr1":[1,2,3,4],"obj1":{"num1":1,"arr2":["val1","val2",true]}}')
result = client.execute_command('JSON.GET', 'ex1:5')
print(result) 

{"str1":"val1","arr1":[1,2,3,4],"obj1":{"num1":1,"arr2":["val1","val2",true]}}


In [11]:
# RETRIEVE SINGLE KEY

client.execute_command('JSON.SET','ex1:6','$','"val"')
result = client.execute_command('JSON.GET','ex1:6', '$')
print(result)

["val"]


In [12]:
# SINGLE PROPERTY FETCH

client.execute_command('JSON.SET','ex1:7', '$', '{"field1": "val1"}')
result = client.execute_command('JSON.GET','ex1:7', '$.field1')
print(result)

["val1"]


In [13]:
# MULTI-PROPERTY FETCH

client.execute_command('JSON.SET','ex1:8', '$', '{"field1": "val1", "field2": "val2"}')
result = client.execute_command('JSON.GET','ex1:8', '$.field1', '$.field2')
print(result)

{"$.field1":["val1"],"$.field2":["val2"]}


In [14]:
# NESTED PROPERTY FETCH

client.execute_command('JSON.SET','ex1:9', '$', '{"obj1": {"str1": "val1", "num2": 2}}')
result = client.execute_command('JSON.GET','ex1:9', '$.obj1.num2')
print(result); 

[2]


In [15]:
# ARRAY FETCH

client.execute_command('JSON.SET','ex1:10', '$', 
        '{"str1": "val1", "str2": "val2", "arr1":[1,2,3,4], "obj1": {"num1": 1,"arr2":["val1","val2", "val3"]}}')
result = client.json().get('ex1:10', '$.obj1.arr2')
print(result)
result = client.json().get('ex1:10', '$.arr1[1]' )
print(result)
result = client.json().get('ex1:10', '$.obj1.arr2[0:2]')
print(result)
result = client.json().get('ex1:10', '$.arr1[-3:]')
print(result)


[['val1', 'val2', 'val3']]
[2]
['val1', 'val2']
[2, 3, 4]


# 1.5 - Efficient Updates

In [16]:
# UPDATE ENTIRE OBJECT

client.execute_command('JSON.SET','ex1:11', '$', '{"field1": "val1"}')
client.execute_command('JSON.SET','ex1:11', '$', '{"FOO": "BAR"}')
result = client.json().get('ex1:11')
print(result)

{'FOO': 'BAR'}


In [17]:
# UPDATE A SINGLE FIELD

client.execute_command('JSON.SET','ex1:11', '$', '{"field1": "val1", "field2": "val2"}')
client.execute_command('JSON.SET','ex1:11', '$.field2', '"NEW-VALUE"')
result = client.json().get('ex1:11')
print(result); 

{'field1': 'val1', 'field2': 'NEW-VALUE'}


In [18]:
# UPDATE A PROPERTY IN AN EMBEDDED JSON DOCUMENT

client.execute_command('JSON.SET','ex1:12', '$', '{"obj1": {"str1": "val1", "NUMBER": 2}}')
client.execute_command('JSON.SET','ex1:12', '$.obj1.NUMBER', 30)
result = client.json().get('ex1:12')
print(result)

{'obj1': {'str1': 'val1', 'NUMBER': 30}}


In [19]:
# UPDATE ARRAY VIA INDEX

client.execute_command('JSON.SET','ex1:13', '$', '{"arr1": ["val1", "val2", "val3"]}')
client.execute_command('JSON.SET','ex1:13', '$.arr1[1]', '"FOO"')
result = client.execute_command('JSON.GET','ex1:13')
print(result) 

{'arr1': ['val1', 'FOO', 'val3']}


In [20]:
# DELETE ENTIRE OBJECT

client.execute_command('JSON.SET','ex1:14', '$', '{"field1": "val1"}')
client.execute_command('JSON.DEL','ex1:14')
result = client.execute_command('JSON.GET','ex1:14')
print(result)

None


In [21]:
# DELETE A SINGLE PROPERTY FROM AN OBJECT

client.execute_command('JSON.SET', 'ex1:15', '$', '{"field1": "val1", "field2": "val2"}')
client.execute_command('JSON.DEL', 'ex1:15', '$.field1') 
client.execute_command('JSON.GET', 'ex1:15')

{'field2': 'val2'}

In [22]:
# DELETE A PROPERTY FROM AN EMBEDDED OBJECT

client.execute_command('JSON.SET', 'ex1:16', '$', '{"obj1": {"str1": "val1", "num2": 2}}')
result=client.execute_command('JSON.GET', 'ex1:16')
print(result)
client.execute_command('JSON.DEL','ex1:16', '$.obj1.num2') 
client.execute_command('JSON.GET', 'ex1:16')

{'obj1': {'str1': 'val1', 'num2': 2}}


{'obj1': {'str1': 'val1'}}

In [23]:
# DELETE A SINGLE ITEM FROM AN ARRAY

client.execute_command('JSON.SET','ex1:17', '$', '{"arr1": ["val1", "val2", "val3"]}')
client.execute_command('JSON.DEL','ex1:17', '$.arr1[0]') 
client.execute_command('JSON.GET','ex1:17')

{'arr1': ['val2', 'val3']}

# __________________________________________________________________________________________

# 2 - Basic Query Operations

In [24]:
# Load Functions Needed
from redis.commands.search.field import NumericField, TagField, TextField, GeoField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query

In [25]:
# LOAD THE DATA

client.json().set('product:15970', '$', {"id": 15970, "gender": "Men", "season":["Fall", "Winter"], "description": "Turtle Neck Men Navy Blue Shirt", "price": 34.95, "city": "Boston", "coords": "-71.057083, 42.361145"})
client.json().set('product:59263', '$', {"id": 59263, "gender": "Women", "season":["Fall", "Winter", "Spring", "Summer"],"description": "Titan Women Silver Watch", "price": 129.99, "city": "Dallas", "coords": "-96.808891, 32.779167"})
client.json().set('product:46885', '$', {"id": 46885, "gender": "Boys", "season":["Fall"], "description": "Ben 10 Boys Navy Blue Slippers", "price": 45.99, "city": "Denver", "coords": "-104.991531, 39.742043"})


True

# 2.1 - Index Creation

In [27]:
# DROP INDEX IF NECESSARY
client.execute_command('FT.DROPINDEX','idx1')

'OK'

In [28]:
# CREATE AN INDEX
idx1_def = IndexDefinition(index_type=IndexType.JSON, prefix=['product:'])
schema = [
    NumericField('$.id', as_name='id'),
    TagField('$.gender', as_name='gender'),
    TagField('$.season.*', as_name='season'),
    TextField('$.description', as_name='description'),
    NumericField('$.price', as_name='price'),
    TextField('$.city', as_name='city'),
    GeoField('$.coords', as_name='coords')
]
result = client.ft('idx1').create_index(schema, definition=idx1_def)
print(result)

OK


# 2.2 - Simple Queries

In [29]:
# QUERY ALL DOCUMENTS FOR THE INDEX

query = Query('*')
result = client.ft('idx1').search(query)
print(result)

Result{3 total, docs: [Document {'id': 'product:59263', 'payload': None, 'json': '{"id":59263,"gender":"Women","season":["Fall","Winter","Spring","Summer"],"description":"Titan Women Silver Watch","price":129.99,"city":"Dallas","coords":"-96.808891, 32.779167"}'}, Document {'id': 'product:15970', 'payload': None, 'json': '{"id":15970,"gender":"Men","season":["Fall","Winter"],"description":"Turtle Neck Men Navy Blue Shirt","price":34.95,"city":"Boston","coords":"-71.057083, 42.361145"}'}, Document {'id': 'product:46885', 'payload': None, 'json': '{"id":46885,"gender":"Boys","season":["Fall"],"description":"Ben 10 Boys Navy Blue Slippers","price":45.99,"city":"Denver","coords":"-104.991531, 39.742043"}'}]}


In [30]:
# SINGLE TERM TEXT 
# FIND ALL DOCUMENTS WITH A CERTAIN WORD IN A TEXT FIELD

query = Query('@description:Slippers')
result = client.ft('idx1').search(query)
print(result)

Result{1 total, docs: [Document {'id': 'product:46885', 'payload': None, 'json': '{"id":46885,"gender":"Boys","season":["Fall"],"description":"Ben 10 Boys Navy Blue Slippers","price":45.99,"city":"Denver","coords":"-104.991531, 39.742043"}'}]}


# 2.3 - More Sophisticated Queries

In [31]:
# EXACT PHRASE TEXT

# Find all documents with a given phrase in a text field.

query = Query('@description:("Blue Shirt")')
result = client.ft('idx1').search(query)
print(result)

Result{1 total, docs: [Document {'id': 'product:15970', 'payload': None, 'json': '{"id":15970,"gender":"Men","season":["Fall","Winter"],"description":"Turtle Neck Men Navy Blue Shirt","price":34.95,"city":"Boston","coords":"-71.057083, 42.361145"}'}]}


In [32]:
# NUMERIC RANGE
# Find all documents with a numeric field in a given range.

query = Query('@price:[40,130]')
result = client.ft('idx1').search(query)
print(result)

Result{2 total, docs: [Document {'id': 'product:59263', 'payload': None, 'json': '{"id":59263,"gender":"Women","season":["Fall","Winter","Spring","Summer"],"description":"Titan Women Silver Watch","price":129.99,"city":"Dallas","coords":"-96.808891, 32.779167"}'}, Document {'id': 'product:46885', 'payload': None, 'json': '{"id":46885,"gender":"Boys","season":["Fall"],"description":"Ben 10 Boys Navy Blue Slippers","price":45.99,"city":"Denver","coords":"-104.991531, 39.742043"}'}]}


In [33]:
# TAG ARRAY (Tags are more economical than text)
# Find all documents that contain a given value in an array field (tag).

query = Query('@season:{Spring}')
result = client.ft('idx1').search(query)
print(result)

Result{1 total, docs: [Document {'id': 'product:59263', 'payload': None, 'json': '{"id":59263,"gender":"Women","season":["Fall","Winter","Spring","Summer"],"description":"Titan Women Silver Watch","price":129.99,"city":"Dallas","coords":"-96.808891, 32.779167"}'}]}


In [34]:
# Logical AND
# Find all documents contain both a numeric field in a range and a word in a text field.

query = Query('@price:[40, 100] @description:Blue')
result = client.ft('idx1').search(query)
print(result)

Result{1 total, docs: [Document {'id': 'product:46885', 'payload': None, 'json': '{"id":46885,"gender":"Boys","season":["Fall"],"description":"Ben 10 Boys Navy Blue Slippers","price":45.99,"city":"Denver","coords":"-104.991531, 39.742043"}'}]}


In [35]:
#Logical OR
#Find all documents that either match tag value or text value.

query = Query('(@gender:{Women})|(@city:Boston)')
result = client.ft('idx1').search(query)
print(result)

Result{2 total, docs: [Document {'id': 'product:59263', 'payload': None, 'json': '{"id":59263,"gender":"Women","season":["Fall","Winter","Spring","Summer"],"description":"Titan Women Silver Watch","price":129.99,"city":"Dallas","coords":"-96.808891, 32.779167"}'}, Document {'id': 'product:15970', 'payload': None, 'json': '{"id":15970,"gender":"Men","season":["Fall","Winter"],"description":"Turtle Neck Men Navy Blue Shirt","price":34.95,"city":"Boston","coords":"-71.057083, 42.361145"}'}]}


In [36]:
# Negation
# Find all documents that do not contain a given word in a text field.

query = Query('-(@description:Shirt)')
result = client.ft('idx1').search(query)
print(result)

Result{2 total, docs: [Document {'id': 'product:59263', 'payload': None, 'json': '{"id":59263,"gender":"Women","season":["Fall","Winter","Spring","Summer"],"description":"Titan Women Silver Watch","price":129.99,"city":"Dallas","coords":"-96.808891, 32.779167"}'}, Document {'id': 'product:46885', 'payload': None, 'json': '{"id":46885,"gender":"Boys","season":["Fall"],"description":"Ben 10 Boys Navy Blue Slippers","price":45.99,"city":"Denver","coords":"-104.991531, 39.742043"}'}]}


In [37]:
# Prefix
# Find all documents that have a word that begins with a given prefix value.

query = Query('@description:Nav*')
result = client.ft('idx1').search(query)
print(result)

Result{2 total, docs: [Document {'id': 'product:15970', 'payload': None, 'json': '{"id":15970,"gender":"Men","season":["Fall","Winter"],"description":"Turtle Neck Men Navy Blue Shirt","price":34.95,"city":"Boston","coords":"-71.057083, 42.361145"}'}, Document {'id': 'product:46885', 'payload': None, 'json': '{"id":46885,"gender":"Boys","season":["Fall"],"description":"Ben 10 Boys Navy Blue Slippers","price":45.99,"city":"Denver","coords":"-104.991531, 39.742043"}'}]}


In [38]:
# Suffix
# Find all documents that contain a word that ends with a given suffix value.

query = Query('@description:*Watch')
result = client.ft('idx1').search(query)
print(result)

Result{1 total, docs: [Document {'id': 'product:59263', 'payload': None, 'json': '{"id":59263,"gender":"Women","season":["Fall","Winter","Spring","Summer"],"description":"Titan Women Silver Watch","price":129.99,"city":"Dallas","coords":"-96.808891, 32.779167"}'}]}


# 2.4 - Fuzzy Queries

In [39]:
# FUZZY SEARCH
# Find all documents that contain a word that is within 1 Levenshtein distance of a given word.

# 'navy' SIMILAR TO 'wavy'

query = Query('@description:%wavy%')
result = client.ft('idx1').search(query)
print(result)

Result{2 total, docs: [Document {'id': 'product:15970', 'payload': None, 'json': '{"id":15970,"gender":"Men","season":["Fall","Winter"],"description":"Turtle Neck Men Navy Blue Shirt","price":34.95,"city":"Boston","coords":"-71.057083, 42.361145"}'}, Document {'id': 'product:46885', 'payload': None, 'json': '{"id":46885,"gender":"Boys","season":["Fall"],"description":"Ben 10 Boys Navy Blue Slippers","price":45.99,"city":"Denver","coords":"-104.991531, 39.742043"}'}]}


# 2.5 - Geo-located Queries

In [40]:

# Find all documents that have geographic coordinates within a given range of a given coordinate. 
# Colorado Springs coords (long, lat) = -104.800644, 38.846127

query = Query('@coords:[-104.800644 38.846127 100 mi]')
result = client.ft('idx1').search(query)
print(result)

Result{1 total, docs: [Document {'id': 'product:46885', 'payload': None, 'json': '{"id":46885,"gender":"Boys","season":["Fall"],"description":"Ben 10 Boys Navy Blue Slippers","price":45.99,"city":"Denver","coords":"-104.991531, 39.742043"}'}]}


# __________________________________________________________________________________________

# 3 - Advanced Querying and JSON


In [41]:
# Data Loading
client.json().set('warehouse:1', '$', {
            "city": "Boston",
            "location": "42.361145, -71.057083",
            "inventory":[{
                "id": 15970,
                "gender": "Men",
                "season":["Fall", "Winter"],
                "description": "Turtle Neck Men Navy Blue Shirt",
                "price": 34.95
            },{
                "id": 59263,
                "gender": "Women",
                "season": ["Fall", "Winter", "Spring", "Summer"],
                "description": "Titan Women Silver Watch",
                "price": 129.99
            },{
                "id": 46885,
                "gender": "Boys",
                "season": ["Fall"],
                "description": 
                "Ben 10 Boys Navy Blue Slippers",
                "price": 45.99
}]})

True

# 3.1 - Array Filtering


In [42]:
# ALL ARRAY PROPERTIES
# Fetch all properties of an array.

result = client.json().get('warehouse:1', Path('$.inventory[*]'))
print(json.dumps(result, indent=4)) 

[
    {
        "id": 15970,
        "gender": "Men",
        "season": [
            "Fall",
            "Winter"
        ],
        "description": "Turtle Neck Men Navy Blue Shirt",
        "price": 34.95
    },
    {
        "id": 59263,
        "gender": "Women",
        "season": [
            "Fall",
            "Winter",
            "Spring",
            "Summer"
        ],
        "description": "Titan Women Silver Watch",
        "price": 129.99
    },
    {
        "id": 46885,
        "gender": "Boys",
        "season": [
            "Fall"
        ],
        "description": "Ben 10 Boys Navy Blue Slippers",
        "price": 45.99
    }
]


In [43]:
# ALL PROPERTIES IN A FIELD
# Get all values of a field within an array.

result = client.json().get('warehouse:1', Path('$.inventory[*].price'))
print(json.dumps(result, indent=4)) 

[
    34.95,
    129.99,
    45.99
]


# 3.2 - Relational Queries


In [44]:
# RELATIONAL - EQUALITY
# Get all items within an array where a text field matches a given value.

result = client.json().get('warehouse:1', Path('$.inventory[?(@.description=="Turtle Neck Men Navy Blue Shirt")]'))
print(json.dumps(result, indent=4))

[
    {
        "id": 15970,
        "gender": "Men",
        "season": [
            "Fall",
            "Winter"
        ],
        "description": "Turtle Neck Men Navy Blue Shirt",
        "price": 34.95
    }
]


In [45]:
# RELATIONAL - LESS THAN
# Get all items within an array where a numeric field is less than a given value.

result = client.json().get('warehouse:1', Path('$.inventory[?(@.price<100)]'))
print(json.dumps(result, indent=4)) 

[
    {
        "id": 15970,
        "gender": "Men",
        "season": [
            "Fall",
            "Winter"
        ],
        "description": "Turtle Neck Men Navy Blue Shirt",
        "price": 34.95
    },
    {
        "id": 46885,
        "gender": "Boys",
        "season": [
            "Fall"
        ],
        "description": "Ben 10 Boys Navy Blue Slippers",
        "price": 45.99
    }
]


In [46]:
# RELATIONAL - GREATER THAN OR EQUAL
# Get all items within an array where a numeric field is greater than or equal to a given value.

result = client.json().get('warehouse:1', Path('$.inventory[?(@.id>=20000)]'))
print(json.dumps(result, indent=4)) 

[
    {
        "id": 59263,
        "gender": "Women",
        "season": [
            "Fall",
            "Winter",
            "Spring",
            "Summer"
        ],
        "description": "Titan Women Silver Watch",
        "price": 129.99
    },
    {
        "id": 46885,
        "gender": "Boys",
        "season": [
            "Fall"
        ],
        "description": "Ben 10 Boys Navy Blue Slippers",
        "price": 45.99
    }
]


# 3.3 - LOGICAL Operators


In [47]:
# Logical AND &&
# Get all items within an array that meet two relational operations.

result = client.json().get('warehouse:1', Path('$.inventory[?(@.gender=="Men"&&@.price>20)]'))
print(json.dumps(result, indent=4)) 

[
    {
        "id": 15970,
        "gender": "Men",
        "season": [
            "Fall",
            "Winter"
        ],
        "description": "Turtle Neck Men Navy Blue Shirt",
        "price": 34.95
    }
]


In [48]:
# Logical OR ||
# Get all items within an array that meet at least one relational operation. In this case, RETURN ONLY THE IDs OF THE ITEMS.

result = client.json().get('warehouse:1', Path('$.inventory[?(@.price<100||@.gender=="Women")].id'))
print(json.dumps(result, indent=4))

[
    15970,
    59263,
    46885
]


# 3.4 - Regular Expressions


In [49]:
# REGEX - Contains Exact
# Fetch all items within an array that match a given regex pattern.

result = client.json().get('warehouse:1', Path('$.inventory[?(@.description =~ "Blue")]'))
print(json.dumps(result, indent=4))

[
    {
        "id": 15970,
        "gender": "Men",
        "season": [
            "Fall",
            "Winter"
        ],
        "description": "Turtle Neck Men Navy Blue Shirt",
        "price": 34.95
    },
    {
        "id": 46885,
        "gender": "Boys",
        "season": [
            "Fall"
        ],
        "description": "Ben 10 Boys Navy Blue Slippers",
        "price": 45.99
    }
]


In [50]:
# REGEX - Contains, Case Insensitive
# Get all items within an array where a field contains a term, case insensitive.

result = client.json().get('warehouse:1', Path('$.inventory[?(@.description =~ "(?i)watch")]'))
print(json.dumps(result, indent=4))

[
    {
        "id": 59263,
        "gender": "Women",
        "season": [
            "Fall",
            "Winter",
            "Spring",
            "Summer"
        ],
        "description": "Titan Women Silver Watch",
        "price": 129.99
    }
]


In [51]:
# REGEX - Begins With
# Get all items within an array where a field begins with a given expression.

result = client.json().get('warehouse:1', Path('$.inventory[?(@.description =~ "^T")]'))
print(json.dumps(result, indent=4))

[
    {
        "id": 15970,
        "gender": "Men",
        "season": [
            "Fall",
            "Winter"
        ],
        "description": "Turtle Neck Men Navy Blue Shirt",
        "price": 34.95
    },
    {
        "id": 59263,
        "gender": "Women",
        "season": [
            "Fall",
            "Winter",
            "Spring",
            "Summer"
        ],
        "description": "Titan Women Silver Watch",
        "price": 129.99
    }
]


# 3.5 - Cross-Document Indexing & Querying

In [52]:
# DROP INDEX IF NECESSARY
client.execute_command('FT.DROPINDEX','cross_idx')

'OK'

In [53]:
# DATA FROM 2 INVENTORY SYSTEMS

client.json().set('prod:123', '$', {"name":"book","price":20})
client.json().set('prod:124', '$', {"name":"chair","price":50})
client.json().set('prod:125', '$', {"name":"desk","price":400})
                  
client.json().set('item:33', '$', {"desc":"pen","cost":5})
client.json().set('item:34', '$', {"desc":"pencil","cost":3})

True

In [54]:
# CREATE AN INDEX ACROSS THE 2 DOCUMENT TYPES
cross_idx_def = IndexDefinition(index_type=IndexType.JSON, prefix=['prod:','item:'])
schema = [
#    TextField('$.name', as_name='name'),
    TextField("$.['name','desc']", as_name='name_desc'),
    NumericField("$.['price','cost']", as_name='price_cost'),
]
result = client.ft('cross_idx').create_index(schema, definition=cross_idx_def)
print(result)

OK


In [55]:
# SEARCH FOR PRODUCTS AND ITEMS NAMED 'book' OR 'pencil':

query = Query('(@name_desc:book | @name_desc:pencil)')
result = client.ft('cross_idx').search(query)
print(result)


Result{2 total, docs: [Document {'id': 'prod:123', 'payload': None, 'json': '{"name":"book","price":20}'}, Document {'id': 'item:34', 'payload': None, 'json': '{"desc":"pencil","cost":3}'}]}


In [56]:
# FIND PRODUCTS AND ITEMS PRICED BETWEEN $5 AND $20

query = Query('@price_cost:[5,20]')
result = client.ft('cross_idx').search(query)
print(result)

Result{2 total, docs: [Document {'id': 'prod:123', 'payload': None, 'json': '{"name":"book","price":20}'}, Document {'id': 'item:33', 'payload': None, 'json': '{"desc":"pen","cost":5}'}]}


In [57]:
#ft.search my-idx '-(@name:pen*)'
# FIND PRODUCTS AND ITEMS WITH NAMES THAT DO NOT BEGIN WITH 'pen'

query = Query('-(@name_desc:pen*)')
result = client.ft('cross_idx').search(query)
print(result)

Result{3 total, docs: [Document {'id': 'prod:123', 'payload': None, 'json': '{"name":"book","price":20}'}, Document {'id': 'prod:124', 'payload': None, 'json': '{"name":"chair","price":50}'}, Document {'id': 'prod:125', 'payload': None, 'json': '{"name":"desk","price":400}'}]}


# END